In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:
data = pd.read_csv('SalesKaggle3.csv')
data.head()

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99


In [3]:
print(data.isnull().sum())

Order                    0
File_Type                0
SKU_number               0
SoldFlag            122921
SoldCount           122921
MarketingType            0
ReleaseNumber            0
New_Release_Flag         0
StrengthFactor           0
PriceReg                 0
ReleaseYear              0
ItemCount                0
LowUserPrice             0
LowNetPrice              0
dtype: int64


In [4]:
print(data.describe())

               Order    SKU_number      SoldFlag     SoldCount  ReleaseNumber  \
count  198917.000000  1.989170e+05  75996.000000  75996.000000  198917.000000   
mean   106483.543242  8.613626e+05      0.171009      0.322306       3.412202   
std     60136.716784  8.699794e+05      0.376519      1.168615       3.864243   
min         2.000000  5.000100e+04      0.000000      0.000000       0.000000   
25%     55665.000000  2.172520e+05      0.000000      0.000000       1.000000   
50%    108569.000000  6.122080e+05      0.000000      0.000000       2.000000   
75%    158298.000000  9.047510e+05      0.000000      0.000000       5.000000   
max    208027.000000  3.960788e+06      1.000000     73.000000      99.000000   

       New_Release_Flag  StrengthFactor       PriceReg    ReleaseYear  \
count     198917.000000    1.989170e+05  198917.000000  198917.000000   
mean           0.642248    1.117115e+06      90.895243    2006.016414   
std            0.479340    1.522090e+06      86.736

In [7]:
data.drop(columns =['MarketingType','File_Type','Order','ReleaseNumber','New_Release_Flag','StrengthFactor'],inplace=True)

In [8]:
data.head()

,SKU_number,SoldFlag,SoldCount,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,1737127,0.0,0.0,44.99,2015,8,28.97,31.84
1,3255963,0.0,0.0,24.81,2005,39,0.00,15.54
2,612701,0.0,0.0,46.00,2013,34,30.19,27.97
3,115883,1.0,1.0,100.00,2006,20,133.93,83.15
4,863939,1.0,1.0,121.95,2010,28,4.00,23.99


In [10]:
X = data.drop(columns =['SoldCount'])
y = data['SoldCount']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
X_test.head()

,SKU_number,SoldFlag,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
50754,193503,0.0,102.00,2006,19,33.94,42.99
150971,137508,NaN,110.00,2005,21,4.76,46.47
124025,679621,NaN,25.95,2008,24,16.57,25.15
15340,560999,1.0,149.33,2007,88,118.61,38.98
153924,606636,NaN,52.95,2008,17,32.34,51.24


In [13]:
df = pd.DataFrame(data)

In [14]:
df['Before2010'] = (df['ReleaseYear'] < 2010).astype(int)

In [15]:
sales_summary = df.groupby('Before2010').agg(
    Avg_LowUserPrice=('LowUserPrice', 'mean'),
    Total_SoldItems=('SoldFlag', 'sum')
).reset_index()

In [16]:
best_sellers = df[df['SoldFlag'] == 1].sort_values(by='LowUserPrice')


In [17]:
print("Sales Summary:")
print(sales_summary)
print("\nBest Sellers based on Lowest Price:")
print(best_sellers[['SKU_number', 'ReleaseYear', 'LowUserPrice', 'SoldFlag']])

Sales Summary:
   Before2010  Avg_LowUserPrice  Total_SoldItems
0           0         31.717498           4787.0
1           1         30.590424           8209.0

Best Sellers based on Lowest Price:
       SKU_number  ReleaseYear  LowUserPrice  SoldFlag
73640      615929         1971          0.00       1.0
55703     2642444         2005          0.00       1.0
7351      1999424         2014          0.00       1.0
55655      106963         2003          0.00       1.0
55592      200913         2009          0.00       1.0
...           ...          ...           ...       ...
5837       871773         2000       1135.02       1.0
49693      616049         2006       1353.99       1.0
64049     2018078         2008       2244.75       1.0
25849      625253         2010       2646.73       1.0
35510      640246         2009       2646.73       1.0

[12996 rows x 4 columns]


In [18]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

NameError: name 'RandomForestRegressor' is not defined

In [19]:
df_encoded = pd.get_dummies(df, columns=['ReleaseYear'], drop_first=True)

In [20]:
print("Sales Summary:")

Sales Summary:


,SKU_number,SoldFlag,SoldCount,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice,Before2010
0,1737127,0.0,0.0,44.99,2015,8,28.97,31.84,0
1,3255963,0.0,0.0,24.81,2005,39,0.00,15.54,1
2,612701,0.0,0.0,46.00,2013,34,30.19,27.97,0
3,115883,1.0,1.0,100.00,2006,20,133.93,83.15,1
4,863939,1.0,1.0,121.95,2010,28,4.00,23.99,0


In [28]:
print(X_train.isnull().sum())  # Check for NaN values in features
  # Check for NaN values in target


SKU_number          0
SoldFlag        98329
PriceReg            0
ReleaseYear         0
ItemCount           0
LowUserPrice        0
LowNetPrice         0
dtype: int64


In [27]:
print(f"X length: {len(X)}, y length: {len(y)}")

X length: 198917, y length: 198917


In [29]:
X_train_cleaned = X_train.dropna()

In [30]:
y_train_cleaned = y_train.loc[X_train_cleaned.index]

In [34]:
model = LinearRegression()
model.fit(X_train_cleaned, y_train_cleaned)

LinearRegression()

In [33]:
X_test_cleaned = X_test.dropna()  # Clean X_test by dropping NaN rows
y_test_cleaned = y_test.loc[X_test_cleaned.index] 


In [35]:
y_pred = model.predict(X_test_cleaned)

In [36]:

mae = mean_absolute_error(y_test_cleaned, y_pred)
mse = mean_squared_error(y_test_cleaned, y_pred)
r2 = r2_score(y_test_cleaned, y_pred)


In [37]:
print("Model Performance:")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

# Step 7: Print predictions on test data
print("\nPredictions on cleaned test set:")
print(y_pred)

Model Performance:
Mean Absolute Error: 0.23763749631460165
Mean Squared Error: 0.9291650669373318
R² Score: 0.36617304197654554

Predictions on cleaned test set:
[-0.0660653   1.946049    0.06821432 ...  0.00253643 -0.09890515
  0.17155646]


In [38]:
X_test = X_test.reset_index(drop=True)

In [39]:
predictions_df = pd.DataFrame(y_pred, columns=['Predicted_SoldCount'])

# Link predictions back to the product details
# For example, let's assume 'SKU_number' is in your test set
output_df = X_test[['SKU_number', 'ReleaseYear', 'PriceReg']].copy()
output_df['Predicted_SoldCount'] = predictions_df['Predicted_SoldCount']

# Display the output
print(output_df)

       SKU_number  ReleaseYear  PriceReg  Predicted_SoldCount
0          193503         2006    102.00            -0.066065
1          137508         2005    110.00             1.946049
2          679621         2008     25.95             0.068214
3          560999         2007    149.33            -0.019289
4          606636         2008     52.95            -0.034780
...           ...          ...       ...                  ...
39779      743561         2004    175.00                  NaN
39780      163892         1995     50.00                  NaN
39781      619776         2010    163.25                  NaN
39782      203357         2001     29.95                  NaN
39783       51863         2006    128.40                  NaN

[39784 rows x 4 columns]


In [40]:
output_df.head()

,SKU_number,ReleaseYear,PriceReg,Predicted_SoldCount
0,193503,2006,102.00,-0.066065
1,137508,2005,110.00,1.946049
2,679621,2008,25.95,0.068214
3,560999,2007,149.33,-0.019289
4,606636,2008,52.95,-0.034780


In [41]:
# Round predictions to nearest whole number (0 or 1)
output_df['Predicted_SoldCount'] = output_df['Predicted_SoldCount'].round()

# Display the updated output
print(output_df.head())


   SKU_number  ReleaseYear  PriceReg  Predicted_SoldCount
0      193503         2006    102.00                 -0.0
1      137508         2005    110.00                  2.0
2      679621         2008     25.95                  0.0
3      560999         2007    149.33                 -0.0
4      606636         2008     52.95                 -0.0


In [43]:
output_df.to_csv('predictions_output.csv', index=False)